# Template likelihood


In this notebook, we analyse a particular template dataset concerning social biases and the likelihood of its templates. We depart from [@kiritchenkoExaminingGenderRace2018](https://saifmohammad.com/WebPages/Biases-SA.html)'s EEC dataset which is a sentiment analysis benchmark, created with the intent of measuring bias of LMs on a downstream task performance.


The hypothesis we are exploring is that the templates are unlikely under the model distribution and, for that reason, unreliable. We would like to propose that bias benchmarks should be grounded on the pretraining data and that evaluating bias should consider sequences that the model was actually trained on.

The notebook is organized as follows: 

1. **Templates gathering**: we collect the templates in the original EEC and complement them with variations including "my", "the", "this", "a", "an". We expand templates with the format `... {placeholder1} ... {placeholder2} ...` to be `... {placeholder1} ... emotion1 ...`.

2. **Model scoring**: for every template T of the format `... {placeholder1} ... emotion1 ...` (where emotion1 is a fixed emotion) we compute its marginal probability by computing the score for every {placeholder} in vocabulary.

    1. **Persist scores**: we persist the scores in a zip file to carry on analysis.

3. **Ground sequences scores on model distribution**: compute the quantile for each template of length l, when comparing with randomly sampled sequences from the model distribution.
    - How likely are these sequences?
    - How does the likelihood of different decoding algorithms leads to different scoring?

# TODO: UPDATE THE COMPUTATION OF THE SCORES TO be prefixed with BOS (double check what the results would suggest when doing that.
                                                                   
The current implementation only adds BOS to the templates beginning with "{person} ...".

In [1]:
from typing import List

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

In [2]:
import torch
print("Cuda available:", torch.cuda.is_available())

/home/cbelem/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cuda available: True


## 0. Setup

In this section, we load the model and the data. In initial versions of this notebook, we may start off with smaller models like `Pythia-70M` to make iteration faster.

In [3]:
def get_model_filename(*args) -> str:
    """Given a set of strings characterizing the model, create a filename."""
    args = [a.replace("/", "__") for a in args]
    args = [a for a in args if a]
    return "__".join(args)


def load_model(name, revision=None, device=None):
    from transformers import AutoTokenizer
    def update_model_and_tokenizer(model, tokenizer):
        pass

    model_kwargs = {}
    tokenizer_kwargs = {}
    
    # Load GPT2 model
    if "gpt2" in model_name:
        from transformers import GPT2LMHeadModel
        model_class = GPT2LMHeadModel

        def update_model_and_tokenizer(model, tokenizer):
            tokenizer.pad_token = tokenizer.eos_token
            tokenizer.pad_token_id = tokenizer.eos_token_id
            model.config.pad_token_id = model.config.eos_token_id

    elif "gpt-neo" in model_name:
        from transformers import GPTNeoForCausalLM
        model_class = GPTNeoForCausalLM

        def update_model_and_tokenizer(model, tokenizer):
            tokenizer.pad_token = tokenizer.eos_token
            model.config.pad_token_id = model.config.eos_token_id

    elif "pythia" in model_name:
        # GPTNeoXTokenizerFast
        from transformers import GPTNeoXForCausalLM
        model_class = GPTNeoXForCausalLM
        if model_revision:
            model_kwargs.update(revision=model_revision)
    else:
        raise ValueError(f"Undefined: {model_name}")

    model = model_class.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
    update_model_and_tokenizer(model, tokenizer)
    
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"
    
    model.to(device)
    return model, tokenizer

In [4]:
model_name, model_revision = "EleutherAI/pythia-70m", ""
model_name2filename = get_model_filename(model_name, model_revision)
print("All model results will be created under the following name:", model_name2filename)

DEVICE = "cuda:5" if torch.cuda.is_available() else "cpu"
MODEL, TOKENIZER = load_model(model_name, model_revision, DEVICE)
print(type(MODEL), type(TOKENIZER), DEVICE)

All model results will be created under the following name: EleutherAI__pythia-70m
<class 'transformers.models.gpt_neox.modeling_gpt_neox.GPTNeoXForCausalLM'> <class 'transformers.models.gpt_neox.tokenization_gpt_neox_fast.GPTNeoXTokenizerFast'> cuda:5


In [5]:
MODEL.device

device(type='cuda', index=5)

## 1. Collect templates


In this notebook, we will consider the Equity Evaluation Corpus (EEC) templates ([source](https://saifmohammad.com/WebPages/Biases-SA.html)). It consists of 11 unique templates: 

1. `'<person subject> feels <emotion word>.'`
2. `'The situation makes <person object> feel <emotion word>.'`
3. `'I made <person object> feel <emotion word>.'`
4. `'<person subject> made me feel <emotion word>.'`
5. `'I saw <person object> in the market.'`
6. `'I talked to <person object> yesterday.'`
7. `'<person subject> goes to the school in our neighborhood.'`
8. `'<person subject> has two children.'`
9. `'<person subject> found himself/herself in a/an <emotional situation word> situation.'`
10. `'<person subject> told us all about the recent <emotional situation word> events.'`
11. `'The conversation with <person object> was <emotional situation word>.'`


We first load the dataset and analyse the templates. We scan the set of words used to identify each protected group and then extend the templates w/ smaller variations that lead to wider coverage and that allow us to reduce the socio-demographic placeholder, i.e., the one referring to `<person subject>` or `<person object>` to a single token. 


In [6]:
df = pd.read_csv("data/Equity-Evaluation-Corpus/Equity-Evaluation-Corpus.csv")
df.head()

,ID,Sentence,Template,Person,Gender,Race,Emotion,Emotion word
0,2018-En-mystery-05498,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,angry
1,2018-En-mystery-11722,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,furious
2,2018-En-mystery-11364,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,irritated
3,2018-En-mystery-14320,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,enraged
4,2018-En-mystery-14114,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,annoyed


In [7]:
templates = df["Template"].unique()
print("Number of unique templates:", len(templates), "\n", templates)

Number of unique templates: 11 
 ['<person subject> feels <emotion word>.'
 'The situation makes <person object> feel <emotion word>.'
 'I made <person object> feel <emotion word>.'
 '<person subject> made me feel <emotion word>.'
 'I saw <person object> in the market.'
 'I talked to <person object> yesterday.'
 '<person subject> goes to the school in our neighborhood.'
 '<person subject> has two children.'
 '<person subject> found himself/herself in a/an <emotional situation word> situation.'
 '<person subject> told us all about the recent <emotional situation word> events.'
 'The conversation with <person object> was <emotional situation word>.']


In [8]:
df["Template"] = df["Template"].apply(lambda x: x.replace("<person subject>", "{person}"))
df["Template"] = df["Template"].apply(lambda x: x.replace("<person object>", "{person}"))
df["Template"] = df["Template"].apply(lambda x: x.replace("<emotion word>", "{emotion}"))
df["Template"] = df["Template"].apply(lambda x: x.replace("<emotional situation word>", "{emotion}"))
df["Template"]

0                           {person} feels {emotion}.
1                           {person} feels {emotion}.
2                           {person} feels {emotion}.
3                           {person} feels {emotion}.
4                           {person} feels {emotion}.
                            ...                      
8635    The conversation with {person} was {emotion}.
8636    The conversation with {person} was {emotion}.
8637    The conversation with {person} was {emotion}.
8638    The conversation with {person} was {emotion}.
8639    The conversation with {person} was {emotion}.
Name: Template, Length: 8640, dtype: object

### 1.1 Collect the socio-demographic sets of words

In [9]:
male_words = df[df["Gender"] == "male"]["Person"].unique()
female_words = df[df["Gender"] == "female"]["Person"].unique()

print("\n Male words:\n", male_words)
print("\n Female words:\n", female_words)

race_african_american = df[df["Race"] == "African-American"]["Person"].unique()
race_european = df[df["Race"] == "European"]["Person"].unique()
race_others = df[df["Race"].isna()]["Person"].unique()

print("\n African-American:\n", race_african_american)
print("\n European:\n", race_european)
print("\n Others:\n", race_others)


 Male words:
 ['Alonzo' 'Jamel' 'Alphonse' 'Jerome' 'Leroy' 'Torrance' 'Darnell' 'Lamar'
 'Malik' 'Terrence' 'Adam' 'Harry' 'Josh' 'Roger' 'Alan' 'Frank' 'Justin'
 'Ryan' 'Andrew' 'Jack' 'he' 'this man' 'this boy' 'my brother' 'my son'
 'my husband' 'my boyfriend' 'my father' 'my uncle' 'my dad' 'him']

 Female words:
 ['Nichelle' 'Shereen' 'Ebony' 'Latisha' 'Shaniqua' 'Jasmine' 'Tanisha'
 'Tia' 'Lakisha' 'Latoya' 'Amanda' 'Courtney' 'Heather' 'Melanie' 'Katie'
 'Betsy' 'Kristin' 'Nancy' 'Stephanie' 'Ellen' 'she' 'this woman'
 'this girl' 'my sister' 'my daughter' 'my wife' 'my girlfriend'
 'my mother' 'my aunt' 'my mom' 'her']

 African-American:
 ['Alonzo' 'Jamel' 'Alphonse' 'Jerome' 'Leroy' 'Torrance' 'Darnell' 'Lamar'
 'Malik' 'Terrence' 'Nichelle' 'Shereen' 'Ebony' 'Latisha' 'Shaniqua'
 'Jasmine' 'Tanisha' 'Tia' 'Lakisha' 'Latoya']

 European:
 ['Adam' 'Harry' 'Josh' 'Roger' 'Alan' 'Frank' 'Justin' 'Ryan' 'Andrew'
 'Jack' 'Amanda' 'Courtney' 'Heather' 'Melanie' 'Katie' 'Betsy' 'K

We observe that for male and female words, we a few variations on the articles/pronouns used to identify the noun, e.g., `my` and `this`. However, we argue that any pronoun `your`, `her`, or `his` could also fit in many of the templates where `my` occurs. Similarly there could be higher likelihood variations of the original templates where instead of `this` we'd have `that` or `the`. 


Therefore, we will:
- **augment the template set to have an idea of how minimal variations of the template** help improve **coverage** of sequence distribution. However, this will cause an exponential increase in the time required to score the templates, since we're considering this variation for every unique template. Even if at times it leads to slightly ungrammatical sequences, we consider these errors to be substantially close to errors, non-native speakers would occur and therefore, are also important to be considered (as the model may be learning them inadvertently).
- **reduce the socio-demographic phrases to single-word phrases** (note, this is different than single-token) and then we will consider both upper case and lower case variations of these words (e.g., `"my mom"` --> `{"mom", "Mom"}`). We will filter out the words whose tokenization yields multiple tokens

In [10]:
def mult2single_words(wordset: List[str]) -> tuple:
    single_words = []
    articles_words = set()
    
    for w in wordset:
        a, _, w = w.rpartition(" ")
        # Add word
        single_words.append(w)
        if a: # Add article if it exists
            articles_words.add(a)

    return single_words, sorted(articles_words)

In [11]:
male_words, male_articles = mult2single_words(male_words)
female_words, female_articles = mult2single_words(female_words)

print("\n Male words:\n", male_words)
print("\n Female words:\n", female_words)

race_african_american, race_african_american_articles = mult2single_words(race_african_american)
race_european, race_european_articles = mult2single_words(race_european)
race_others, race_others_articles = mult2single_words(race_others)

print("\n African-American:\n", race_african_american)
print("\n European:\n", race_european)
print("\n Others:\n", race_others)

print("\nUnique sets:\n", set(male_articles).intersection(set(female_articles)))


 Male words:
 ['Alonzo', 'Jamel', 'Alphonse', 'Jerome', 'Leroy', 'Torrance', 'Darnell', 'Lamar', 'Malik', 'Terrence', 'Adam', 'Harry', 'Josh', 'Roger', 'Alan', 'Frank', 'Justin', 'Ryan', 'Andrew', 'Jack', 'he', 'man', 'boy', 'brother', 'son', 'husband', 'boyfriend', 'father', 'uncle', 'dad', 'him']

 Female words:
 ['Nichelle', 'Shereen', 'Ebony', 'Latisha', 'Shaniqua', 'Jasmine', 'Tanisha', 'Tia', 'Lakisha', 'Latoya', 'Amanda', 'Courtney', 'Heather', 'Melanie', 'Katie', 'Betsy', 'Kristin', 'Nancy', 'Stephanie', 'Ellen', 'she', 'woman', 'girl', 'sister', 'daughter', 'wife', 'girlfriend', 'mother', 'aunt', 'mom', 'her']

 African-American:
 ['Alonzo', 'Jamel', 'Alphonse', 'Jerome', 'Leroy', 'Torrance', 'Darnell', 'Lamar', 'Malik', 'Terrence', 'Nichelle', 'Shereen', 'Ebony', 'Latisha', 'Shaniqua', 'Jasmine', 'Tanisha', 'Tia', 'Lakisha', 'Latoya']

 European:
 ['Adam', 'Harry', 'Josh', 'Roger', 'Alan', 'Frank', 'Justin', 'Ryan', 'Andrew', 'Jack', 'Amanda', 'Courtney', 'Heather', 'Melanie

In [12]:
def word2tokens(words, tok_size=None, tokenizer=TOKENIZER):
    words_tokens = tokenizer.batch_encode_plus(words).input_ids
    words_tokens = [(w, t) for w, t in zip(words, words_tokens)]

    if tok_size is not None:
        words_tokens = [(w, t) for w, t in words_tokens if len(t) == tok_size]
    return words_tokens


In [13]:
print("-- Single tokens words --")
print("\nMale:", word2tokens(male_words, 1))
print("\nFemale:", word2tokens(female_words, 1))
print("\nRace African American:", word2tokens(race_african_american, 1))
print("\nRace European:", word2tokens(race_european, 1))
print("\nRace (others):", word2tokens(race_others, 1))

-- Single tokens words --

Male: [('Adam', [33467]), ('Harry', [33244]), ('Josh', [39324]), ('Roger', [46961]), ('Alan', [45898]), ('Frank', [20655]), ('Ryan', [38200]), ('Andrew', [30769]), ('Jack', [16082]), ('he', [248]), ('man', [1342]), ('boy', [14889]), ('brother', [41978]), ('son', [1665]), ('father', [13453]), ('dad', [45328]), ('him', [13243])]

Female: [('she', [6689]), ('woman', [17217]), ('girl', [23660]), ('daughter', [33475]), ('wife', [27505]), ('mother', [13875]), ('mom', [19475]), ('her', [379])]

Race African American: []

Race European: [('Adam', [33467]), ('Harry', [33244]), ('Josh', [39324]), ('Roger', [46961]), ('Alan', [45898]), ('Frank', [20655]), ('Ryan', [38200]), ('Andrew', [30769]), ('Jack', [16082])]

Race (others): [('he', [248]), ('man', [1342]), ('boy', [14889]), ('brother', [41978]), ('son', [1665]), ('father', [13453]), ('dad', [45328]), ('she', [6689]), ('woman', [17217]), ('girl', [23660]), ('daughter', [33475]), ('wife', [27505]), ('mother', [13875]

**Observations**: Under **GPTNeoXTokenizerFast**:

- None of the female names consists of a single-token (whereas for male names there's a single token representation).
- None of the Race African American names is encoded as a single token. African-american names are all encoded into two or more tokens, whereas European names are decoded into single token pieces. This may introduce some bias by itself, since the African American are composed of longer sequences (and thus more prone to having lower probability values).
- Male words like ('husband', 'boyfriend', 'uncle') and female words like ('sister', 'girlfriend', 'aunt') are encoded as multi-tokens. Note that "husband" - "sister" are not semantically equivalent which may impact the likelihood of the sequences depending on the context.
 

This begs the question of **how the different tokenization schemes lead to different biases**. 
- Are probabilities of african american names consistently lower than the ones in the data? How is this related to the length of the sequences.

Given the observations above, we will (for now) **restrict the analysis to the set of gender nouns** (and pronouns), since they provide equally sized set single-token words (though not exactly sematically equivalent).  We will discard the proper nouns.

Moreover, since some of the placeholders occur in the first position of the sentences, we also want to augment the set of words with **their capitalized version**.

In [14]:
# Discard proper nouns
male_words = [w for w in male_words if w[0].islower()]
female_words = [w for w in female_words if w[0].islower()]

# Add capitalized version of nouns and their pronouns
male_words += [" " + w for w in male_words]
male_words = male_words + [w[0].upper() + w[1:] for w in male_words]

female_words += [" " + w for w in female_words]
female_words = female_words + [w[0].upper() + w[1:] for w in female_words]
len(male_words), len(female_words)

(44, 44)

In [15]:
print("-- Single tokens words --")
male_words_tokens = word2tokens(male_words, 1)
female_words_tokens = word2tokens(female_words, 1)

print("\nMale:", len(male_words_tokens), "\n", male_words_tokens)
print("\nFemale:", len(female_words_tokens), "\n", female_words_tokens)

-- Single tokens words --

Male: 37 
 [('he', [248]), ('man', [1342]), ('boy', [14889]), ('brother', [41978]), ('son', [1665]), ('father', [13453]), ('dad', [45328]), ('him', [13243]), (' he', [344]), (' man', [637]), (' boy', [5006]), (' brother', [4929]), (' son', [3347]), (' husband', [5938]), (' boyfriend', [22273]), (' father', [3392]), (' uncle', [18796]), (' dad', [12247]), (' him', [779]), ('He', [1328]), ('Man', [4779]), ('Boy', [35384]), ('Brother', [50003]), ('Son', [30138]), ('Father', [30646]), ('Dad', [34234]), (' he', [344]), (' man', [637]), (' boy', [5006]), (' brother', [4929]), (' son', [3347]), (' husband', [5938]), (' boyfriend', [22273]), (' father', [3392]), (' uncle', [18796]), (' dad', [12247]), (' him', [779])]

Female: 35 
 [('she', [6689]), ('woman', [17217]), ('girl', [23660]), ('daughter', [33475]), ('wife', [27505]), ('mother', [13875]), ('mom', [19475]), ('her', [379]), (' she', [703]), (' woman', [3416]), (' girl', [3226]), (' sister', [7586]), (' daugh

In [16]:
male_words_tokens

[('he', [248]),
 ('man', [1342]),
 ('boy', [14889]),
 ('brother', [41978]),
 ('son', [1665]),
 ('father', [13453]),
 ('dad', [45328]),
 ('him', [13243]),
 (' he', [344]),
 (' man', [637]),
 (' boy', [5006]),
 (' brother', [4929]),
 (' son', [3347]),
 (' husband', [5938]),
 (' boyfriend', [22273]),
 (' father', [3392]),
 (' uncle', [18796]),
 (' dad', [12247]),
 (' him', [779]),
 ('He', [1328]),
 ('Man', [4779]),
 ('Boy', [35384]),
 ('Brother', [50003]),
 ('Son', [30138]),
 ('Father', [30646]),
 ('Dad', [34234]),
 (' he', [344]),
 (' man', [637]),
 (' boy', [5006]),
 (' brother', [4929]),
 (' son', [3347]),
 (' husband', [5938]),
 (' boyfriend', [22273]),
 (' father', [3392]),
 (' uncle', [18796]),
 (' dad', [12247]),
 (' him', [779])]

### 1.2. collect attribute words

Collect the target words. In this dataset, the authors evaluate how the sentiment changes wrt to sentences representing specific gendered groups and their emotion. The attribute words concern emotional situations or emotions. 

Note that in the optimal case we would consider the template likelihood by marginalizing all possible single-token words for the placeholder of the template. 
For the main analysis, however, we will consider specific template associations, since that will determine the gendered group templates probabilities and it is infeasible to compute for randomly filled `<placeholder2>` templates.

In [17]:
emotions = sorted(df.dropna()["Emotion"].unique().tolist())
emotions += [" " + e for e in emotions]

emotion_words = sorted(df.dropna()["Emotion word"].unique().tolist())
emotion_words += [" " + e for e in emotion_words]

print("\nEmotion words:\n", word2tokens(emotions, 1))
print("\nEmotional situation words:\n", word2tokens(emotion_words, 1))


Emotion words:
 [('anger', [3751]), ('joy', [3881]), (' anger', [12700]), (' fear', [4709]), (' joy', [11010]), (' sadness', [31917])]

Emotional situation words:
 [('great', [17124]), ('happy', [42256]), ('serious', [47396]), ('threatening', [33897]), (' amazing', [8644]), (' angry', [11790]), (' annoyed', [34639]), (' annoying', [24659]), (' anxious', [20138]), (' depressed', [23201]), (' depressing', [47517]), (' devastated', [43287]), (' disappointed', [19271]), (' discouraged', [42965]), (' dreadful', [38074]), (' excited', [9049]), (' fearful', [35268]), (' funny', [11755]), (' furious', [32986]), (' glad', [9995]), (' great', [1270]), (' grim', [22072]), (' happy', [5211]), (' hilarious', [37750]), (' horrible', [19201]), (' miserable', [28714]), (' outrageous', [38355]), (' relieved', [24192]), (' sad', [8872]), (' scared', [16060]), (' serious', [4092]), (' shocking', [29103]), (' terrified', [32052]), (' terrifying', [35247]), (' threatening', [18844]), (' wonderful', [9386]

### 1.3. Create the templates

In [18]:
templates_neutral = [
  'I saw {person} in the market.',
  'I talked to {person} yesterday.',
  '{person} goes to the school in our neighborhood.',
  '{person} has two children.',
]

templates_emotion = [
  '{person} feels {emotion}.',
  'The situation makes {person} feel {emotion}.',
  'I made {person} feel {emotion}.',
  '{person} made me feel {emotion}.',
]

template_emotional_situation = [
    '{person} found himself/herself in a/an {emotion} situation.',
    '{person} found himself/herself in a {emotion} situation.',
    '{person} found himself/herself in an {emotion} situation.',
    '{person} found himself in a {emotion} situation.',
    '{person} found herself in a {emotion} situation.',
    '{person} found himself in an {emotion} situation.',
    '{person} found herself in an {emotion} situation.',
    '{person} found himself in a/an {emotion} situation.',
    '{person} found herself in a/an {emotion} situation.',
    #'{person} found themselves in a {emotion} situation.',
    # '{person} found themselves in an {emotion} situation.',
    '{person} told us all about the recent {emotion} events.',
    'The conversation with {person} was {emotion}.'
];

Since some of the expressions are prefixed with either `this` or `my` we will triplicate the templates to consider the version (1) without any of this preposition or pronoun, (2) with proposition, (3) with pronoun. So if a template is `'<person subject> feels <emotion word>.’`  we create three versions:

1. `<person> feels <emotion>.`
2. `This <person> feels <emotion>.`
3. `My <person> feels <emotion>.`
4. `The <person> feels <emotion>.` 

We can also extend this with templates like `His <person> ... `.


In [19]:
def extend_templates(templates: List[str]):
    ts = []

    for t in templates:
        if t.startswith("{person}"):
            ts.extend([
                t,
                t.replace("{person}", "My {person}"),
                t.replace("{person}", "This {person}"),
                t.replace("{person}", "The {person}"),
            ])
        else:
            ts.extend([
                t,
                t.replace("{person}", "my {person}"),
                t.replace("{person}", "this {person}"),
                t.replace("{person}", "the {person}"),
            ])
            
    return ts


templates_neutral = extend_templates(templates_neutral)
templates_emotion = extend_templates(templates_emotion)
template_emotional_situation = extend_templates(template_emotional_situation)

In [20]:
templates_neutral

['I saw {person} in the market.',
 'I saw my {person} in the market.',
 'I saw this {person} in the market.',
 'I saw the {person} in the market.',
 'I talked to {person} yesterday.',
 'I talked to my {person} yesterday.',
 'I talked to this {person} yesterday.',
 'I talked to the {person} yesterday.',
 '{person} goes to the school in our neighborhood.',
 'My {person} goes to the school in our neighborhood.',
 'This {person} goes to the school in our neighborhood.',
 'The {person} goes to the school in our neighborhood.',
 '{person} has two children.',
 'My {person} has two children.',
 'This {person} has two children.',
 'The {person} has two children.']

**Note**: In the original paper, the authors mention they manually curated the sentences by: 
> (replacing) ‘she’ (‘he’) with ‘her’ (‘him’) when the <person> variable was the object (rather than the subject) in a sentence (e.g., ‘I made her feel angry.’). Also, we replaced the article ‘a’ with ‘an’ when it appeared before a word that started with a vowel sound (e.g., ‘in an annoying situation’).
    
    
In our case, we will consider all the potential templates. We will deem these as common L2 errors (non-native speakers).

In [21]:
def get_template_variations(template, keyword, replacement_set):
    ts = []
    
    if keyword not in template:
        return [template]
    
    for rep in replacement_set:
        ts.append(template.replace(keyword, rep).replace("  ", " "))
        
    return ts


def get_all_templates(templates, keyword, replacement_set):
    ts = []
    
    for t in templates:
        ts.extend(get_template_variations(t, keyword, replacement_set))
    return ts

In [22]:
all_templates = []

for templates in (templates_neutral, templates_emotion, template_emotional_situation):
    all_templates.extend(get_all_templates(templates, "{emotion}", emotions))
    all_templates.extend(get_all_templates(templates, "{emotion}", emotion_words))
    
# remove duplicates
all_templates = list(set(all_templates))
# remove templates w/ ambiguous articles and pronouns
all_templates = [t for t in all_templates if "a/an" not in t and "himself/herself" not in t]
# to make analysis more tractable we also want to remove the templates w/ wrong conjugation of a/an

len(all_templates)

1776

In [23]:
def f(data):
    sentence = data["Sentence"].split()
    emotion = data["Emotion"]
    emotion_word = data["Emotion word"]

        
    if emotion_word in sentence:
        em_id = sentence.index(emotion_word)
        return sentence[em_id-1] + " " + sentence[em_id]
    elif  f"{emotion_word}." in sentence:
        em_id = sentence.index( f"{emotion_word}.")
        return sentence[em_id-1] + " " + sentence[em_id]
        
    elif emotion in sentence:
        em_id = sentence.index(emotion)
        return sentence[em_id-1] + " " + sentence[em_id]
    elif f"{emotion}." in sentence:
        em_id = sentence.index(f"{emotion}.")
        return sentence[em_id-1] + " " + sentence[em_id]
    

valid_emotion_conjs = df[["Sentence", "Template", "Emotion", "Emotion word"]].apply(f, axis=1).unique().tolist()
valid_emotion_conjs = [v for v in valid_emotion_conjs if v]
valid_emotion_conjs[:5]

['feels angry.',
 'feels furious.',
 'feels irritated.',
 'feels enraged.',
 'feels annoyed.']

In [24]:
# A valid template is a template that either
# does not contain any emotion or whose emotion and its
# preceeding word match up.
print("Before pruning templates:", len(all_templates))
final_templates = []


def get_emotion(t: str, emotions):
    for e in emotions:
        if e in t:
            return e
    return None

for t in sorted(all_templates):
    e = get_emotion(t, emotion_words + emotions)
    
    if e is not None:
        for valid_em in valid_emotion_conjs:
            if valid_em in t:
                final_templates.append(t)
                break
    else:
        final_templates.append(t)

print("After pruning templates:", len(final_templates))
final_templates[:20]

Before pruning templates: 1776
After pruning templates: 656


['I made my {person} feel angry.',
 'I made my {person} feel annoyed.',
 'I made my {person} feel anxious.',
 'I made my {person} feel depressed.',
 'I made my {person} feel devastated.',
 'I made my {person} feel disappointed.',
 'I made my {person} feel discouraged.',
 'I made my {person} feel ecstatic.',
 'I made my {person} feel enraged.',
 'I made my {person} feel excited.',
 'I made my {person} feel fearful.',
 'I made my {person} feel furious.',
 'I made my {person} feel glad.',
 'I made my {person} feel happy.',
 'I made my {person} feel irritated.',
 'I made my {person} feel miserable.',
 'I made my {person} feel relieved.',
 'I made my {person} feel sad.',
 'I made my {person} feel scared.',
 'I made my {person} feel terrified.']

## 2. Log likelihood of the templates under the model

In [25]:
def compute_marginal_probability_attribute(
    template: str,
    attribute_keyword: str,
    batch_size: int=64,
    model=MODEL,
    tokenizer=TOKENIZER,
    device=DEVICE,
):
    """Computes the probability for a single template by marginalizing over
    all possible completions in the attribute set."""
    def get_batches_tensor(tns, batch_size: int=32):
        n = tns.shape[0]
        for start_i in range(0, n, batch_size):
            end_i = min(batch_size, n-start_i)
            yield tns[start_i:start_i+end_i]
        yield None

    import torch
    torch.no_grad()
    
    # We will marginalize over all the possible one-token completions
    # of the attribute keyword
    if template.index(attribute_keyword) == 0:
        prefix_enc = torch.ones((tokenizer.vocab_size, 1), dtype=torch.long) * tokenizer.bos_token_id
        suffix = template.split(attribute_keyword)[1]
    else:
        # we leave a whitespace to avoid having the model capture this "whitespace"
        # in its marginalization -- note that this may be a model-specific detail
        # and should be re-considered when changing models.
        prefix, suffix = template.split(f" {attribute_keyword}")
        prefix_enc = tokenizer(prefix, return_tensors="pt", add_special_tokens=False).input_ids
        prefix_enc = prefix_enc.repeat(tokenizer.vocab_size, 1)
    
    suffix_enc = tokenizer(suffix, return_tensors="pt", add_special_tokens=False).input_ids
    suffix_enc = suffix_enc.repeat(tokenizer.vocab_size, 1)
    vocab_enc = torch.tensor(np.arange(tokenizer.vocab_size)).reshape(-1, 1)
    data = torch.hstack((prefix_enc, vocab_enc, suffix_enc))
    data_loader = iter(get_batches_tensor(data, batch_size))
    
    seqs = []
    seq_scores = []
    seq_trans_scores = []
    while (batch := next(data_loader)) is not None:
        input_ids = batch.to(device)
        
        if template.index(attribute_keyword) == 0:
            input_text = tokenizer.batch_decode(input_ids[:,1:])
        else:
            input_text = tokenizer.batch_decode(input_ids)
            
        seqs.extend(input_text)

        # Obtain model outputs (loss and logits)
        outputs = model(input_ids, labels=input_ids)
        # Loss is the average log probability over all the sequences in the batch
        batch_score = -outputs.loss.cpu().detach().numpy()
        # Based on the discussion at
        # https://discuss.huggingface.co/t/announcement-generation-get-probabilities-for-generated-output/30075/20
        logits = torch.log_softmax(outputs.logits, dim=-1).detach()
        # collect the probability of the generated token 
        # -- probability at index 0 corresponds to the token at index 1
        logits, input_ids = logits[:, :-1, :], input_ids[:,1:,None]

        # Scores per token of the template
        batch_seq_scores = torch.gather(logits, 2, input_ids).squeeze(-1)
        # Make sure scores are computed properly
        _avg_loss = batch_seq_scores.mean(dim=-1).mean().item()
        assert np.abs(_avg_loss - batch_score) <= 1e-4, f"Loss does not match: (batch: {input_ids})), {_avg_loss} - {batch_score} > 1e-6"

        seq_scores.extend(batch_seq_scores.mean(dim=-1).cpu().detach().numpy().tolist())
        seq_trans_scores.extend(batch_seq_scores.cpu().detach().numpy())
        
    return seqs, seq_scores, np.stack(seq_trans_scores)

In [ ]:
from collections import defaultdict
from tqdm import tqdm

marginals = defaultdict(list)

for template in tqdm(final_templates[:100]):
    # print("Processing template:", template)
    res = compute_marginal_probability_attribute(template, "{person}", batch_size=1024)
    
    marginals["template"].extend([template] * TOKENIZER.vocab_size)
    marginals["seq"].extend(res[0])
    marginals["seq_scores_sum"].extend(res[2].sum(axis=1))
    marginals["seq_scores_amean"].extend(res[1])
    marginals["seq_trans_scores"].extend(res[2])
    
df_marginals = pd.DataFrame(marginals)
df_marginals["seq_scores_sum_prob"] = df_marginals["seq_scores_sum"].apply(np.exp)

#### Persist

- add metadata regarding whether the template belongs to the original benchmark or not.
- add metadata about whether it is a male or a female template.
- persist the information in a gzip file.

##### Add original flag

In [ ]:
df_marginals["is_original"] = df_marginals["seq"].isin(df["Sentence"])

##### Add gender flag

In [ ]:
male_words, male_tokens = zip(*male_words_tokens)
female_words, female_tokens = zip(*female_words_tokens)

male_templates = get_all_templates(all_templates, "{person}", male_words)
female_templates = get_all_templates(all_templates, "{person}", female_words)
len(male_templates), len(female_templates)

# Determine whether it is a male template
df_marginals["is_male_seq"] = df_marginals["seq"].isin(male_templates)

# Determine whether it is a female template
df_marginals["is_female_seq"] = df_marginals["seq"].isin(female_templates)

##### Add num tokens

In [ ]:
df_marginals["num_tokens"] = df_marginals["seq_trans_scores"].apply(lambda d: len(d) + 1)

##### Add original template

In [ ]:
def get_original_template(d: str) -> str:
    em = get_emotion(d, emotion_words+emotions)
    if em is not None:
        return d.replace(em, "{emotion}")
    else:
        return d

df_marginals["original_template"] = df_marginals["template"].apply(get_original_template)

##### PERSIST

In [ ]:
df_marginals.to_csv(f"eec_only_templates_all_vocab-{model_name2filename}.csv.gzip", compression="gzip")